# Lab 4: Trees and Forests

In this assignment, you will explore tree-based classification models to implement predictions for broadband deployment, using a combination of available datasets from the Census and the Federal Communications Commission (FCC).

You will work with these datasets to understand the nature of broadband deployment across Chicago, as well as what features turn out to be good predictors of broadband deployment. The existence of connectivity in a certain area can also be measured in a variety of ways (e.g., available speed tiers, subscriptions, measured performance), and reported in different ways (e.g., by ISPs, by citizens/subscribers). You will explore how reporting and characteristics vary across different datasets and how this may affect your model.

## Learning Objectives

In this assignment, you will learn the following:
* The mechanisms of decision trees and bootstrapping methods, including Random Forest classifiers
* How to work with various broadband connectivity data (ACS, FCC, etc.)
* How to apply tree-based classification models to data (decision tree, Random Forest)
* How to take advantage of various characteristics of Random Forest models, such as feature importance
* How to evaluate classification models using various metrics and approaches 

## Part 1: Tree-Based Models by Hand

In this section, you will manually implement a decision tree using entropy and information gain. 
Please do not use code in your calculations for this section. You may either type your solutions using Markdown, or attach a scanned copy of your hand written solutions in your Canvas file submission. 

For this section, you will predict the user experience when streaming a video, based on three attributes: the speed that they have purchased from their ISP, the quality of their WiFi Router, and the type of device they are using to stream.

| ISP Speed | WiFi Router | Device Type | User Experience |
|----------|------------|-----------|----------------|
| Fast    | Regular    | Old     | Low            |
| Medium   | Regular    | Average | High           |
| Slow    | Premium    | New     | Low            |
| Fast    | Premium    | New     | High           |
| Fast    | Premium    | Average | Low            |
| Medium   | Regular    | Old     | High           |
| Slow    | Regular    | Old     | High           |
| Slow    | Premium    | Average | Low            |
| Slow    | Premium    | Average | High           |
| Slow    | Regular    | Average | High           |

### 1.1 Calculating Entropy

What is the entropy for the User Experience attribute? Were you expecting a result around this number? Why?

*YOUR ANSWER HERE*

### 1.2 Calculating Information Gain

What is the information gain on User Experience if you split the observations on the WiFi Router attribute?

*YOUR ANSWER HERE*

### 1.3 Building a Simple Decision Tree


Construct a simple two-level decision tree that can be used to predict User Experience. Use information gain as your splitting criterion. Feel free to use a greedy approach, recursively splitting on the attribute that gives the greatest information gain at each step.

Draw your final tree and include an image of it with your final submission.

*YOUR ANSWER HERE*

## Part 2: Loading and Exploring Datasets 

### 2.1 FCC Broadband Map Data

In the first part of the lab, you will compile the datasets that you will use in your analysis. Given that the FCC dataset is quite large, we will work with a truncated version of the data for Chicago. We have also included the original source in case you wish to explore the full data yourself. 

The FCC makes its data for broadband maps available on [its website](https://broadbandmap.fcc.gov). Data is also [available for download](https://broadbandmap.fcc.gov/#/data-download). Internet service providers are required to fill out a "Form 477", which reports service offerings in each Census Block.  

The specific data that we will use for this lab is from June 2019 in Cook County, IL. We provide the the following code for downloading the data, according to [the API documentation](https://dev.socrata.com/foundry/opendata.fcc.gov/sgz3-kiqt). You should have 794,141 rows. The download should take no more than 5 minutes.

In [1]:
import pandas as pd
from sodapy import Socrata

client = Socrata("opendata.fcc.gov", None)

# GEOID is the only geographic attribute, so we limit the data to Cook County 
# Returned as JSON from API / converted to Python list of dictionaries by sodapy 
results = client.get("sgz3-kiqt", limit=800000, where="starts_with(blockcode, '17031')")

# Convert to pandas DataFrame
fcc_df = pd.DataFrame.from_records(results)

print(fcc_df.shape)
fcc_df.head(2)

(794141, 17)


,logrecno,provider_id,frn,providername,dbaname,holdingcompanyname,hoconum,hocofinal,stateabbr,blockcode,techcode,consumer,maxaddown,maxadup,business,maxcirdown,maxcirup
0,23542275,50860,0003729340,CBTS Technology Solutions LLC,CBTS Technology Solutions LLC,Cincinnati Bell Inc.,130254,Cincinnati Bell Inc.,IL,170318017022031,50,0,0,0,1,0,0
1,23543612,50860,0003729340,CBTS Technology Solutions LLC,CBTS Technology Solutions LLC,Cincinnati Bell Inc.,130254,Cincinnati Bell Inc.,IL,170318030121010,30,0,0,0,1,0,0


First, produce the following three maps for the City of Chicago:
1. The maximum contractual downstream speed offered by any provider in each Census block group.
2. The number of unique ISPs that offer service in each Census block group.
3. The number of unique ISPs that offer service at or above 25 Mbps downstream and 3 Mbps upstream in each Census block group. (This is the FCC's definition of broadband Internet access, which you can read about more in the [2019 broadband deployment report](https://docs.fcc.gov/public/attachments/FCC-19-44A1.pdf)). Use contractual downstream and upstream speed.

**Note:** Geographic boundaries are only directly available on the Chicago Open Data Portal at the Census block and Census tract level. However, the GeoPandas `dissolve` command can be used to obtain boundaries at the Census block group level. Recall that Census block groups correspond to 12 digits of the FIPS code based on the [Census geography hierarchy](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html). 

In [2]:
# YOUR CODE HERE 

In [3]:
# MAP 1 HERE 

In [4]:
# MAP 2 HERE 

In [5]:
# MAP 3 HERE 

### 2.2 ACS Data

The American Community Survey (ACS) provides data on broadband Internet access subscriptions, as reported by participants. For this lab, we will use the ACS 5-year estimates at the Census block group level for 2018.

Use the Census API to perform the following for block groups in the City of Chicago:
1. Load ACS data for the following field: 
    * the percentages of broadband Internet access of any type
2. Load ACS data for Total Population, White, Black, and Median Income – and then compute the following: 
    * the percentage of each Census block group's population that is White and Black; 
    * the median income for the block gloup; 
    * the population density of the block group (e.g. in units of population per square kilometer) 
    
Present your answer as a dataframe showing only the first few rows. Some of these computations should be familiar from the previous lab. We've provided code using the `censusdata` package for pulling the necessary ACS data. 

In [6]:
import censusdata

# Useful for finding the ACS tables you want 
# censusdata.search('acs5', 2018, 'label', 'broadband') 
# censusdata.printtable(censusdata.censustable('acs5', 2018, 'B28011'))

# Pull ACS data 
census_tables = {
    'GEO_ID': 'GEO_ID', 
    'B28011_001E': 'Internet Total', 
    'B28011_004E': 'Broadband', 
    'B02001_001E': 'Race Total', 
    'B02001_002E': 'White', 
    'B02001_003E': 'Black', 
    'B19013_001E': 'Median Income'}

acs_df = censusdata.download("acs5", 
                              2018, 
                              censusdata.censusgeo([("state", "17"), 
                                                    ("county", "031"),
                                                    ("tract", "*"),
                                                    ("block group", "*")]), 
                              list(census_tables.keys()))

# Rename columns 
acs_df.rename(columns=census_tables, inplace=True)

print(acs_df.shape)
acs_df.head(2)

(3993, 7)


,GEO_ID,Internet Total,Broadband,Race Total,White,Black,Median Income
"Block Group 1, Census Tract 8088, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:808800> block group:1",1500000US170318088001,788,690,1738,1538,70,96111
"Block Group 2, Census Tract 8088, Cook County, Illinois: Summary level: 150, state:17> county:031> tract:808800> block group:2",1500000US170318088002,601,516,1994,1672,39,173646


In [7]:
# YOUR CODE HERE 

## Part 3: Prediction with Tree-Based Models

In this part of the assignment, you will use the data and features that you computed to develop tree-based prediction models for broadband Internet deployment at the Census block group level. 

### 3.1 Decision Trees

#### 3.1.1 Training Decison Trees

First, train a decision tree classifier to predict if a Census block group has broadband Internet access or not (i.e., at least one ISP that offers service at or above 25 Mbps downstream and 3 Mbps upstream). Tune your classifier with a hyperparameter grid and use k-fold cross validation. 

In the previous lab, you were asked to construct hyperparameter grids and manually send data through the pipeline. For this lab, you may use scikit-learn's inbuilt `Pipeline` and `GridSearchCV` objects to simplify this step. 

Some additional notes on training:

- Use `random_state=0` when both splitting your data and training your classifiers.
- Given the size of this dataset, it will be sufficient to forgo the validation set and only separate your data into training and testing sets (i.e., use `sklearn.model_selection.test_train_split`)
- Your classifier should use the following features:
    - The ACS population characteristics from Section 2.2 (percentage of population that is White and Black, median income, population density)
    - The number of ISPs serving that block group
- Tune your classifier with a hyperparameter grid, using the following hyperparameter options:
    - `criterion`: `gini` or `entropy`
    - `max_depth`: 1, 3, 5
    - `min_samples_split`: 2, 5, 10
- Use K-fold cross validation with `k = 10`, using accuracy as your scoring metric.
- For evaluation metrics, print accuracy, precision, and recall on the test set for each hyperparameter combination.

Print a dataframe summarizing your grid search results. This should have one row for each classifier that you trained (18 total rows) specifying the hyperparamters for that classifier along with its evaluation metrics averaged across the folds (accuracy, precision, and recall). 

In [8]:
# YOUR CODE HERE 

#### 3.1.2 Evaluation I: Interpreting Trees

Print the tree with the highest average test accuracy. Note that `sklearn.tree.export_graphviz` may be helpful. 

In [9]:
# YOUR CODE HERE

What was the most important feature in the tree that yielded the highest accuracy? 

In [10]:
# YOUR CODE HERE

#### 3.1.3 Evaluation II: Confusion Matrices and Precision-Recall Curves

For all questions in this section, use the decision tree from above that yielded the highest average accuracy. 

Report the confusion matrix for this decision tree. Then use the output of this confusion matrix to manually calculate the following:

1. What is the test precision for this tree?
2. What is the test recall for this tree?
3. What is the test F1 score for this tree? 

After manually computing these metrics, use the built-in `scikit-learn.metrics` functions to verify your answers. 

In [11]:
# YOUR CODE HERE 

*YOUR ANSWER HERE*

Plot the precision-recall curve for this decision tree. Then answer the following questions:

1. Describe what you observe in the precision-recall curve in plain language.
2. Does it make sense for precision to decrease as recall increases? Why or why not? 
3. Can you identify a 'sweet spot' in the graph? (i.e., a threshold that balances nicely precision and recall)

In [12]:
# YOUR CODE HERE 

*YOUR ANSWER HERE*

#### 3.1.4 Subsetting Features
Note that the number of ISPs is closely (and directly) related to the predicted outcome. Suppose you no longer have access to the FCC data. Re-run your decision tree classification using just the ACS demographic features. How does this model perform compared to the model including the number of ISPs as a predictor? 

In [13]:
# YOUR CODE HERE 

*YOUR ANSWER HERE*

### 3.2 Random Forests

#### 3.2.1 Training Random Forests

Next, train a Random Forest classifier to predict broadband Internet deployment. As before, tune your classifier with a hyperparameter grid and use k-fold cross validation. Again, print a dataframe summarizing the classifiers, the hyperparameters for each classifier, and their respective evaluation metrics.

As with the previous section: 
- You may use scikit-learn's inbuilt `Pipeline` and `GridSearchCV` functions if you prefer.
- Use `random_state=0` when both splitting your data and training your classifiers.
- Given the size of this dataset, it will be sufficient to forgo the validation set and only separate your data into training and testing sets (i.e., use `sklearn.model_selection.test_train_split`)
- Your classifier should use the following features:
    - The ACS population characteristics from Section 2.2 (percentage of population that is White and Black, median income, population density)
    - The number of ISPs serving that block group
- Tune your classifier with a hyperparameter grid, using the following hyperparameter options (note the differences for Random Forests):
    - `n_estimators`: 100, 1000, 5000
    - `criterion`: `gini` or `entropy`
    - `max_depth`: 1, 3, 5
    - `min_samples_split`: 2, 5, 10
- Use K-fold cross validation with `k = 10`, using accuracy as your scoring metric.
- For evaluation metrics, print accuracy, precision, and recall on the test set for each hyperparameter combination.

**Note:** This will take noticeably longer than training the single decision tree, but should not exceed 40 minutes. We highly recommend creating a smaller sample of this data to work with while you test your code. 

In [14]:
# YOUR CODE HERE 

#### 3.2.2 Evaluation I: Interpreting Random Forests

Identify the Random Forest with the highest average test accuracy. Create a plot showing feature importance. 

In [15]:
# YOUR CODE HERE 

#### 3.2.3 Evaluation II: Confusion Matrices and Precision Recall Curves

For all questions in this section, use the Random Forest classifier that yielded the highest average accuracy. 

Report the confusion matrix for this Random Forest classifier using the test data. Then manually calculate the following (and feel free to then check your work with the built-in functions):

1. What is the test precision for this classifier?
2. What is the test recall for this classifier?
3. What is the test F1 score for this classifier? 

How do these figures compare to the test metrics for the decision tree?

In [16]:
# YOUR CODE HERE 

*YOUR ANSWER HERE*

Plot the precision-recall curve for this Random Forest. On the same axes, plot the precision-recall curve for the decision tree that you previously plotted in Section 3.1.3. How do they compare? Which would you choose if you wanted to maximize precision? What if you wanted to maximize recall?

In [17]:
# YOUR CODE HERE 

*YOUR ANSWER HERE*

### 3.3 Policy Applications and Model Selection

Imagine you have been hired by the City of Chicago, who is administering a survey about broadband quality. However, they do not know which block groups have broadband and must choose a model to predict broadband deployment. 

1. It is important for you to reach as many block groups with broadband as possible without visiting every single block group. Of the classifiers you trained above, which should you use to predict broadband deployment in this situation? Why?
2. Budget cuts have struck! Priorities have shifted, and you are now told that you can only survey a small number of block groups. You are not worried about representativeness, and your main goal is to avoid wasting resources by visiting block groups that do not have broadband. Which classifier do you use now? Why?
3. You find out only after training that your population density data was in units of people per square mile, not people per square kilometer, meaning that your data would need to be rescaled to be properly interpreted. Do you now need to retrain your model with the rescaled data to get an updated set of predictions? Why or why not?
4. The City of Chicago is interested in better understanding the predictive features in your model. Why do you think the number of ISP providers and population density are important predictors? 

There is a deep policy debate around broadband access related to the role of public investment, public-private-partnerships, and subsidies. This includes a discussion of 'rate-of-return' carriers and what ISPs should be allowed to charge for broadband access in rural areas. For further reading, see [this discussion](https://www.usac.org/high-cost/resources/rules-orders/rate-of-return-reform-order/) on rate-of-return reform and this [FCC briefing](https://docs.fcc.gov/public/attachments/FCC-19-77A1.pdf). 

*YOUR ANSWER HERE*

### 3.4 Working with Thresholds

So far, we've been using GridSearch and Cross-Validation (`GridSearchCV`) to find the best models. Internally, this method is running `predict` on each validation fold, computing the evaluation metrics (accuracy, precision, etc.) for each fold and returning the average for the metrics. Recall that `predict` uses 0.5 as the default classification threshold, meaning every observation with a probability score above 0.5 is classified as True.

In many cases, `predict` is not ideal. Instead, we might want to choose a classification threshold ourselves, rather than use the default 0.5 cutoff. The threshold might be a fixed value (say every observation with a probability above 0.7 receives a predicted label of True). Alternatively, the threshold might be based on a percentage of the total observations (e.g. the top 10% of observations with the highest probability scores will be labelled as True, while the rest will be labelled as False). Which approach to choose will depend on your policy problem. In this exercise, we will explore the effect of using custom thresholds through both approaches. 

Use the best Random Forest classifier identified in the previous section to output prediction probabilities for your testing dataset. Note that you will need to use `predict_proba` rather than `predict` to get these probability scores. Be sure that you understand what the output of `predict_proba` returns and its dimensions. Then, compute precision and recall for different thresholds. 

#### 3.4.1 Thresholds I: Fixed Values Approach
Use the following fixed values to convert probability scores (the output of `predict_proba`) into binary predictions: 0, 0.3, 0.5, 0.7, 0.9. Output a table summarizing the precision and recall at each of these thresholds. 

In [18]:
# YOUR CODE HERE 

#### 3.4.2 Thresholds II: Percentage Approach

Now, use the percentage-based approach using the following threshold percentages: 1%, 5%, 10%, 20%, 50% 100%. 

Using the 1% threshold, for example, you would need to output probability scores for each observation, sort these probabilities in descending order based on this score, and then assign the top 1% True (and the bottom 99% False). If your dataset had 1,000 observations, that would mean that just 10 observations would be marked True. 

Again, output a table summarizing the precision and recall at each of these thresholds. 

In [19]:
# YOUR CODE HERE 

#### 3.4.3 Summarizing Thresholds

Describe what you observed in the two summary tables above. How do precision and recall change when adjusting the thresholds using the two approaches? Are these results consistent with your precision-recall curves? Discuss a situation when using a threshold approach would make sense when evaluating classification models. 

*YOUR ANSWER HERE*